d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px; height: 163px">
</div>

# Streaming Deployment

After batch deployment, continuous model inference using a technology like Spark Streaming represents the second most popular deployment option.  This lesson introduces Spark Streaming and how to perform inference on a stream of incoming data.

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) In this lesson you:<br>
 - Make predictions on streaming data
 - Connect to a Spark Stream
 - Predict using an `sklearn` model on a stream of data
 - Stream predictions into an always up-to-date parquet file

-sandbox
### Inference on Streaming Data

Spark Streaming enables...<br><br>

* Scalable and fault-tolerant operations that continuously perform inference on incoming data
* Streaming applications can also incorporate ETL and other Spark features to trigger actions in real time

This lesson is meant as an introduction to streaming applications as they pertain to production machine learning jobs.  

Streaming poses a number of specific obstacles. These obstacles include:<br><br>

* *End-to-end reliability and correctness:* Applications must be resilient to failures of any element of the pipeline caused by network issues, traffic spikes, and/or hardware malfunctions
* *Handle complex transformations:* applications receive many data formats that often involve complex business logic
* *Late and out-of-order data:* network issues can result in data that arrives late and out of its intended order
* *Integrate with other systems:* Applications must integrate with the rest of a data infrastructure

-sandbox
Streaming data sources in Spark...<br><br>

* Offer the same DataFrames API for interacting with your data
* The crucial difference is that in structured streaming, the DataFrame is unbounded
* In other words, data arrives in an input stream and new records are appended to the input DataFrame

<div><img src="https://files.training.databricks.com/images/eLearning/ETL-Part-3/structured-streamining-model.png" style="height: 400px; margin: 20px"/></div>

Spark is a good solution for...<br><br>

* Batch inference
* Incoming streams of data

For low-latency inference, however, Spark may or may not be the best solution depending on the latency demands of your task

Run the following cell to set up our environment.

In [6]:
%run "./Includes/Classroom-Setup"

-sandbox
### Connecting to the Stream

As data technology matures, the industry has been converging on a set of technologies.  Apache Kafka and the Azure managed alternative Event Hubs have become the ingestion engine at the heart of many pipelines.  

This technology brokers messages between producers, such as an IoT device writing data, and consumers, such as a Spark cluster reading data to perform real time analytics. There can be a many-to-many relationship between producers and consumers and the broker itself is scalable and fault tolerant.

We'll simulate a stream using the `maxFilesPerTrigger` option.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/>  There are a number of ways to stream data.  One other common design pattern is to stream from an Azure Blob Container where any new files that appear will be read by the stream.

Import the dataset in Spark.

In [9]:
airbnbDF = spark.read.parquet("/mnt/training/airbnb/sf-listings/airbnb-cleaned-mlflow.parquet/")

display(airbnbDF)

Create a schema for the data stream.

In [11]:
from pyspark.sql.types import DoubleType, IntegerType, StructType

schema = (StructType()
.add("host_total_listings_count", DoubleType())
.add("neighbourhood_cleansed", IntegerType())
.add("zipcode", IntegerType())
.add("latitude", DoubleType())
.add("longitude", DoubleType())
.add("property_type", IntegerType())
.add("room_type", IntegerType())
.add("accommodates", DoubleType())
.add("bathrooms", DoubleType())
.add("bedrooms", DoubleType())
.add("beds", DoubleType())
.add("bed_type", IntegerType())
.add("minimum_nights", DoubleType())
.add("number_of_reviews", DoubleType())
.add("review_scores_rating", DoubleType())
.add("review_scores_accuracy", DoubleType())
.add("review_scores_cleanliness", DoubleType())
.add("review_scores_checkin", DoubleType())
.add("review_scores_communication", DoubleType())
.add("review_scores_location", DoubleType())
.add("review_scores_value", DoubleType())
.add("price", DoubleType())
)

Check to make sure the schemas match.

In [13]:
schema == airbnbDF.schema

Check the number of shuffle partitions.

In [15]:
spark.conf.get("spark.sql.shuffle.partitions")

Change this to 8.

In [17]:
spark.conf.set("spark.sql.shuffle.partitions", "8")

Create a data stream using `readStream` and `maxFilesPerTrigger`.

In [19]:
streamingData = (spark
                 .readStream
                 .schema(schema)
                 .option("maxFilesPerTrigger", 1)
                 .parquet("/mnt/conor-work/airbnb/airbnb-cleaned-mlflow.parquet")
                 .drop("price"))

### Apply an `sklearn` Model on the Stream

Using the DataFrame API, Spark allows us to interact with a stream of incoming data in much the same way that we did with a batch of data.

Import a `spark_udf`

In [22]:
import mlflow
import mlflow.sklearn
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

with mlflow.start_run(run_name="Final RF Model") as run: 
  df = pd.read_csv("/dbfs/mnt/training/airbnb/sf-listings/airbnb-cleaned-mlflow.csv")
  X = df.drop(["price"], axis=1)
  y = df["price"]

  rf = RandomForestRegressor(n_estimators=100, max_depth=5)
  rf.fit(X, y)
  
  mlflow.sklearn.log_model(rf, "random-forest-model")
  
  runID = run.info.run_uuid
  experimentId = run.info.experiment_id

In [23]:
import mlflow.pyfunc

pyfunc_udf = mlflow.pyfunc.spark_udf(spark, "random-forest-model", run_id=runID)

Transform the stream with a prediction.

In [25]:
predictionsDF = streamingData.withColumn("prediction", pyfunc_udf(*streamingData.columns))

display(predictionsDF)

### Write out a Stream of Predictions

You can perform writes to any target database.  In this case, write to a parquet file.  This file will always be up to date, another component of an application can query this endpoint at any time.

In [27]:
checkpointLocation = userhome + "/academy/stream.checkpoint"
writePath = userhome + "/academy/predictions"

(predictionsDF
  .writeStream                                           # Write the stream
  .format("delta")                                       # Use the delta format
  .partitionBy("zipcode")                                # Specify a feature to partition on
  .option("checkpointLocation", checkpointLocation)      # Specify where to log metadata
  .option("path", writePath)                             # Specify the output path
  .outputMode("append")                                  # Append new records to the output path
  .start()                                               # Start the operation
)

Take a look at the underlying file.  Refresh this a few times.

In [29]:
spark.read.format("delta").load(writePath).count()

Stop the stream.

In [31]:
[q.stop() for q in spark.streams.active]

Things to note:<br><br>

* For batch processing, you can trigger a stream every 24 hours to maintain state
* You can easily combine historic and new data in the same stream

## Review

**Question:** What are commonly approached as data streams?  
**Answer:** Apache Kafka and the Azure managed alternative Event Hubs are common data streams.  Additionally, it's common to monitor a directory for incoming files.  When a new file appears, it is brought into the stream for processing.

**Question:** How does Spark ensure exactly-once data delivery and maintain metadata on a stream?  
**Answer:** Checkpoints give Spark this fault tolerance through the ability to maintain state off of the cluster.

**Question:** How does the Spark approach to streaming integrate with other Spark features?  
**Answer:** Spark Streaming uses the same DataFrame API, allowing easy integration with other Spark functionality.

## Next Steps

Start the next lesson, [Real Time Deployment]($./08-Real-Time-Deployment ).

## Additional Topics & Resources

**Q:** Where can I find out more information on streaming ETL jobs?  
**A:** Check out the Databricks blog post <a href="https://databricks.com/blog/2017/01/19/real-time-streaming-etl-structured-streaming-apache-spark-2-1.html" target="_blank">Real-time Streaming ETL with Structured Streaming in Apache Spark 2.1</a>

**Q:** Where can I get more information on integrating Streaming and Kafka?  
**A:** Check out the <a href="https://spark.apache.org/docs/latest/structured-streaming-kafka-integration.html" target="_blank">Structured Streaming + Kafka Integration Guide</a>

**Q:** Where can I see a case study on an IoT pipeline using Spark Streaming?  
**A:** Check out the Databricks blog post <a href="https://databricks.com/blog/2017/04/26/processing-data-in-apache-kafka-with-structured-streaming-in-apache-spark-2-2.html" target="_blank">Processing Data in Apache Kafka with Structured Streaming in Apache Spark 2.2</a>

-sandbox
&copy; 2019 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>